In [ ]:
import gpmultipy, scipy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from gpmultipy import dataset
from gpmultipy.model import Model
from gpmultipy.prior import Prior
from gpmultipy.kernel import RBF, White
from gpmultipy.freeze import Freezer
from gpmultipy.sampler.slice import Slice
from gpmultipy.kernel import Product, Linear

In [ ]:
%matplotlib inline

In [ ]:
ds = dataset.DataSet('data/')
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='F',effects=['Well'])
y = np.log2(y)
y = y-y[0,:]
y = (y-y.mean())/y.std()

dm = np.ones((y.shape[1],1))

In [ ]:
def wellPosition(number):
    number = int(number)
    if number > 100:
        return wellPosition(number%100)
    if number == 0:
        return 9,9
    return [(number-1) % 10, (number-1)/10]

position = np.array(map(wellPosition,labels[0]))
zip(labels[0],position)

In [ ]:
select = np.all(position != 0,1)


In [ ]:
yKernel = White(1,.1)
k1 = RBF(1,1,1)

# yKernel = White(1,1)
# k1 = RBF(1,1,1)

p = y.shape[1]
dm = np.ones((1,p))

prior = Prior(x,k1,range(dm.shape[0]))

model = Model(x,y,dm)

In [ ]:
ySigmaSlice = Slice('ySigma',
                    lambda x: model.dataLikelihood(yKernel,sigma=x),
                    #lambda x: scipy.stats.uniform(1e-6,1e0).logpdf(x),
                    #lambda x: scipy.stats.lognorm(1,loc=1e-2,scale=1e-1).logpdf(x),
                    lambda x: scipy.stats.lognorm(1,scale=1e-0).logpdf(x),
                    #debug_lognorm,
                    .2,5,logspace=True)

kSigmaSlice = Slice('kSigma',
                    lambda x: prior.loglikelihood(model.beta,sigma=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    #debug_unif,
                    .2,5,logspace=True)

kLengthscaleSlice = Slice('kLengthscale',
                    lambda x: prior.loglikelihood(model.beta,lengthscale=x),
                    lambda x: scipy.stats.uniform(1e-2,1e2).logpdf(x),
                    .2,5,logspace=True)

samples = []
freeze = Freezer(yKernel=yKernel,k1=k1,model=model)

In [ ]:
thin = 10
burnin = 0
nsample = 1000

for i in range(nsample):
    prior.sample(model,yKernel)

    k1.sigma = kSigmaSlice.sample(k1.sigma)
    yKernel.sigma = ySigmaSlice.sample(yKernel.sigma)
    k1.lengthscale = kLengthscaleSlice.sample(k1.lengthscale)

    if i % thin == 0 and i > burnin:
        print model.dataLikelihood(yKernel),yKernel.sigma, k1.sigma, k1.lengthscale
        samples.append(freeze.freeze())

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(231)
plt.plot(x,y)

plt.subplot(232)
plt.plot(x,np.array([s['model']['beta'][:,0] for s in samples]).T,c='r',alpha=.5)

plt.subplot(234)
plt.hist(np.log10([s['yKernel']['sigma'] for s in samples]))

plt.subplot(235)
plt.hist(np.log10([s['k1']['sigma'] for s in samples]))

plt.subplot(236)
plt.hist(np.log10([s['k1']['lengthscale'] for s in samples]))

In [ ]:
mu = np.array([s['model']['beta'][:,0] for s in samples]).T

plt.plot(x,mu.std(1)*2);

plt.twinx()
plt.plot(x,abs(mu.mean(1)),c='k');

In [ ]:
plt.imshow(mu,interpolation='none')

In [ ]:
plt.figure(figsize=(6,4))

# gpfanova.plot.plotFunctionSamples(m_noReplicate.functionSamples(0).values,x=x[:,0])
plt.plot(x[:,0],y,c='k',alpha=.4)

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/ura3-pq-replicate/model-noReplicate-singleReplicate.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(6,4))

mu = samples[-1]['model']['beta'][:,0]
cov = yKernel.K(x)
ysample = mu + scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]),cov,size=model.r)

plt.plot(x[:,0],ysample.T,c='k',alpha=.4); 
plt.plot(x[:,0],y,c='b',alpha=.4); 
# plt.plot(x[:,0],fsample,c='r',lw=3);

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(np.arange(.2,1,.2),fontsize=20)
plt.yticks(np.arange(-2,2,.5),fontsize=20)

plt.savefig("figures/model-noEffect-generative.pdf",bbox_inches='tight',dpi=300)